In [5]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset

In [6]:
target_dataset = 'ogbn-arxiv'
# This will download the ogbn-arxiv to the 'networks' folder
dataset = PygNodePropPredDataset(name=target_dataset, root='data')

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:08<00:00,  9.25it/s]
Processing...


Extracting data\arxiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<?, ?it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<?, ?it/s]

Saving...



Done!


In [7]:
data = dataset[0]
print(data)

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343, 1])


In [19]:
import netCDF4 as nc
import pandas as pd

In [7]:
fn = 'data\\tas_scenario_245\\tas_mon_mod_ssp245_192_000.nc'
ds = nc.Dataset(fn)

In [22]:
ds.variables

{'height': <class 'netCDF4._netCDF4.Variable'>
 float64 height()
     standard_name: height
     long_name: height
     units: m
     positive: up
     axis: Z
 unlimited dimensions: 
 current shape = ()
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float32 lat(lat)
     standard_name: latitude
     long_name: latitude
     units: degrees_north
     axis: Y
 unlimited dimensions: 
 current shape = (144,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float32 lon(lon)
     standard_name: longitude
     long_name: longitude
     units: degrees_east
     axis: X
 unlimited dimensions: 
 current shape = (192,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'tas': <class 'netCDF4._netCDF4.Variable'>
 float32 tas(time, lat, lon)
     standard_name: air_temperature
     long_name: Near-Surface Air Temperature
     units: K
     coordinates: height
     _F

In [29]:
h = ds.variables['tas']
times = ds.variables['time']
jd = nc.num2date(times[:],times.units)
hs = pd.DataFrame(h[:,0],index=jd)
df = hs
df

,0,1,2,3,4,5,6,7,8,9,...,182,183,184,185,186,187,188,189,190,191
1850-01-16 12:00:00,252.504318,252.397537,252.378250,252.359940,252.339279,252.322586,252.311447,252.291946,252.263794,252.238846,...,252.595215,252.599319,252.598465,252.590942,252.585617,252.588089,252.591873,252.595535,252.595993,252.596390
1850-02-15 00:00:00,236.253250,236.135452,236.129105,236.117264,236.102219,236.080032,236.051392,236.016647,235.993729,235.976318,...,235.972702,235.986206,236.019287,236.049271,236.083862,236.144577,236.188065,236.224197,236.247482,236.308228
1850-03-16 12:00:00,217.723312,217.455887,217.431107,217.364426,217.300583,217.280472,217.259201,217.231644,217.209274,217.164795,...,216.997879,217.072556,217.169479,217.294357,217.404343,217.515503,217.577103,217.749802,217.885910,217.923996
1850-04-16 00:00:00,213.946365,213.532303,213.524582,213.490433,213.460251,213.444748,213.434982,213.424606,213.404831,213.378052,...,212.971848,213.122849,213.277267,213.475220,213.665359,213.646805,213.730179,213.986160,214.073318,214.239914
1850-05-16 12:00:00,210.434448,209.970627,209.917358,209.852737,209.807663,209.785843,209.775223,209.744019,209.717529,209.707474,...,209.635178,209.715988,209.838913,209.990112,210.149857,210.200241,210.218307,210.456619,210.635666,210.762527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100-08-16 12:00:00,215.557297,215.196823,215.263596,215.225952,215.195923,215.151688,215.120682,215.084839,215.051392,215.007614,...,214.807205,214.939865,215.098434,215.238571,215.301437,215.405396,215.521042,215.610229,215.837402,216.011475
2100-09-16 00:00:00,215.710129,215.325317,215.349487,215.326294,215.290161,215.246216,215.223709,215.204834,215.168869,215.111893,...,214.878128,215.021362,215.151566,215.324417,215.436157,215.462479,215.675415,215.832520,215.849411,216.010910
2100-10-16 12:00:00,227.069580,226.931473,226.953079,226.911255,226.868118,226.816818,226.774414,226.730347,226.684128,226.655029,...,226.654053,226.706223,226.747635,226.800781,226.861328,226.920776,226.968918,227.031296,227.090378,227.172806
2100-11-16 00:00:00,249.893433,249.785522,249.772385,249.756714,249.729538,249.706055,249.685257,249.670731,249.646606,249.615891,...,250.018356,250.015381,250.012329,250.007645,250.008957,250.012619,250.002884,249.993851,249.986160,249.983810


In [27]:
times.units

'days since 1850-01-01'

In [9]:
for dim in ds.dimensions.values():
    print(dim)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 144
<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 192
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 3012


In [10]:
for var in ds.variables.values():
    print(var)

<class 'netCDF4._netCDF4.Variable'>
float64 height()
    standard_name: height
    long_name: height
    units: m
    positive: up
    axis: Z
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y
unlimited dimensions: 
current shape = (144,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (192,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float32 tas(time, lat, lon)
    standard_name: air_temperature
    long_name: Near-Surface Air Temperature
    units: K
    coordinates: height
    _FillValue: 1e+20
    missing_value: 1e+20
    comment: near-surface (u

In [12]:
prcp = ds['tas'][:]

In [13]:
prcp

masked_array(
  data=[[[252.50432, 252.39754, 252.37825, ..., 252.59554, 252.596  ,
          252.59639],
         [252.54451, 252.59258, 252.5957 , ..., 252.50557, 252.51363,
          252.50642],
         [252.94832, 252.91284, 252.89702, ..., 253.13135, 253.06746,
          253.0024 ],
         ...,
         [239.60498, 239.67603, 239.74731, ..., 239.55318, 239.54456,
          239.56294],
         [238.38908, 238.39507, 238.42204, ..., 238.48787, 238.42256,
          238.37598],
         [238.22913, 238.21057, 238.17973, ..., 238.252  , 238.24617,
          238.23369]],

        [[236.25325, 236.13545, 236.1291 , ..., 236.2242 , 236.24748,
          236.30823],
         [234.15779, 234.04053, 234.02185, ..., 234.49792, 234.4449 ,
          234.34554],
         [234.3073 , 234.15503, 234.02246, ..., 234.95537, 234.73547,
          234.50887],
         ...,
         [237.00764, 237.02177, 237.04875, ..., 237.04431, 237.0606 ,
          237.03137],
         [236.92346, 236.9067 , 236.

In [17]:
import xarray as xr
ds = xr.open_dataset('data/tas_scenario_245/tas_mon_mod_ssp245_192_000.nc')
ds

TypeError: 'numpy._DTypeMeta' object is not subscriptable